In [1]:
NODE_COUNT = 16
LOG_DIR = f"/home/<User>/repos/csg_is/calculating_scores/DFL/FULL/fedstellar_DFL_18_07_2024_13_38_39/metrics/participant_"
date_str = "18/07/2024 11:38:39		" # minus two hours for timezone

In [2]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(NODE_COUNT):
    # Directory containing your event files
    log_dir = LOG_DIR +f"{i}"
    print(log_dir)

    date_obj = datetime.strptime(date_str.rstrip(" ").rstrip("	"), "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

/home/<User>/repos/csg_is/calculating_scores/DFL/FULL/fedstellar_DFL_18_07_2024_13_38_39/metrics/participant_0
   step     value                     timestamp         tag
0  2707  0.248649 2024-07-18 11:50:41.541990144  Train/Loss
1  2757  0.172227 2024-07-18 11:50:44.283612160  Train/Loss
2  2807  0.321973 2024-07-18 11:50:46.707164672  Train/Loss
3  2857  0.291531 2024-07-18 11:50:49.548043776  Train/Loss
4  2907  0.176446 2024-07-18 11:50:51.901226240  Train/Loss
2024-07-18 11:57:53 2024-07-18 11:38:56
                step     value                     timestamp
tag                                                         
Test/Accuracy   6544  0.704391 2024-07-18 11:57:44.528952576
Test/F1Score    6544  0.665379 2024-07-18 11:57:44.528992512
Test/Loss       6544  0.872496 2024-07-18 11:57:44.528879360
Test/Precision  6544  0.683934 2024-07-18 11:57:44.528971008
Test/Recall     6544  0.704391 2024-07-18 11:57:44.528982016
/home/<User>/repos/csg_is/calculating_scores/DFL/FULL/fedstell

In [3]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,6544,0.704391,2024-07-18 11:57:44.528952576,participant_0
Test/F1Score,6544,0.665379,2024-07-18 11:57:44.528992512,participant_0
Test/Loss,6544,0.872496,2024-07-18 11:57:44.528879360,participant_0
Test/Precision,6544,0.683934,2024-07-18 11:57:44.528971008,participant_0
Test/Recall,6544,0.704391,2024-07-18 11:57:44.528982016,participant_0
...,...,...,...,...
Test/Accuracy,6590,0.796664,2024-07-18 11:57:44.206026240,participant_15
Test/F1Score,6590,0.780336,2024-07-18 11:57:44.206055936,participant_15
Test/Loss,6590,0.544010,2024-07-18 11:57:44.205992192,participant_15


In [4]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_36644/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,6581.5625,0.7286
Test/F1Score,6581.5625,0.6792
Test/Loss,6581.5625,1.6639
Test/Precision,6581.5625,0.6895
Test/Recall,6581.5625,0.7286


In [5]:
concat_df.groupby("tag").std().round(2)

/tmp/ipykernel_36644/582865284.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").std().round(2)


,step,value
tag,,
Test/Accuracy,12.65,0.09
Test/F1Score,12.65,0.10
Test/Loss,12.65,1.31
Test/Precision,12.65,0.10
Test/Recall,12.65,0.09


In [6]:
test_results = concat_df.groupby("tag").mean().round(4)
test_results[test_results.index.str.startswith("Test/")]

/tmp/ipykernel_36644/1227319542.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_results = concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,6581.5625,0.7286
Test/F1Score,6581.5625,0.6792
Test/Loss,6581.5625,1.6639
Test/Precision,6581.5625,0.6895
Test/Recall,6581.5625,0.7286
